In [1]:
#Initialize dependencies
import tensorflow as tf
import os
BASE_FOLDER = '/Users/oblivisheee/Documents/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
%store -r
from modules.transformer_custom import Transformer
from modules.regularization import RegularizedDenseLayer
from modules.data_preprocess import DataGenerator


/Users/oblivisheee/miniconda3/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
train_russian_dir = 'dataset/train/russian'
train_english_dir = 'dataset/train/english'
valid_russian_dir = 'dataset/valid/russian'
valid_english_dir = 'dataset/valid/english'

data_gen = DataGenerator(train_russian_dir, train_english_dir, valid_russian_dir, valid_english_dir)
train_russian_data, train_english_data, valid_russian_data, valid_english_data = data_gen.generate()

train_dataset = tf.data.Dataset.from_tensor_slices((train_russian_data, train_english_data))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_russian_data, valid_english_data))



Train Russian data info: 1 samples
Train English data info: 1 samples
Valid Russian data info: 1 samples
Valid English data info: 1 samples


# Config training

In [3]:
# Configure the Transformer layer

embedding_layer = tf.keras.layers.Embedding(8500, 512)
regularized_layer = RegularizedDenseLayer(512)

model = Transformer(num_layers=6, d_model=512, num_heads=8, dff=2048,
                          input_vocab_size=8500, target_vocab_size=8500, pe_input=10000,
                          pe_target=10000, rate=0.1, embedding=embedding_layer, regularized_layer=regularized_layer)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='auto')
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001,
                                     epsilon=1e-8, amsgrad=True,
                                     beta_1=0.9, beta_2=0.999)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'), 
           tf.keras.metrics.MeanSquaredError(name='mean_squared_error'), 
           tf.keras.metrics.Precision(thresholds=0.5, name='precision'),]


model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)

# Start training

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
session_name = 'novelsdreamer-test'
epochs = 10
model.fit_model(inp_dataset=train_english_data, tar_dataset=train_russian_data, valid_inp_dataset=valid_english_data, valid_tar_dataset=valid_russian_data, epochs=epochs,
                         session_name=session_name,
                         batch_size=64, gradient_accumulation_steps=5,
                         shuffle=True,
                         save_model_each_epoch=True,
                         model_name='novelsdreamer-ru-t4m')

KeyError: 'logs_path'